# Everyone should use Pandas! (and sometimes astropy tables)
* This tutorial is meant to get you started with using Pandas and Astropy tables to organize and work with your data sets 

* Storing your data as pandas dataframes makes querying and sorting your data, merging different data sets, doing statistics, and visualiziing your data faster, easier, and cleaner. 
* Pandas also handels messy data and data with missing fields very well 
* Pandas is a python package widely used to build and work with dataframes. Astropy has a class called Tables which has similar functionality but much more limited and less supported. I introduce it breifly here because it has a few astronomy specific features absent in Pandas that are sometimes useful. For most things the two objects can be used interchangably. 

## Main documentation for both Pandas, Astropy tables, and Astroquery
* [Pandas](http://pandas.pydata.org/pandas-docs/stable/)
* [Astropy Tables](http://docs.astropy.org/en/stable/table/)
* [Astroquery](https://astroquery.readthedocs.io/en/latest/)

In [ ]:
#import all needed python packages
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from astropy import units as u
from astropy.coordinates import SkyCoord

#comment out the next two lines if you do not have astroquery installed
from astroquery.sdss import SDSS
from astroquery.vizier import Vizier

from astropy.table import Table, Column, join
import pandas as pd

## Types of Tables in Pandas
1. Series - 1D array, can be build out of a python dictionary 
2. Dataframes - 2D array 

### Tables are different from numpy arrays:
* Each row in a table is assigned an index. 
* In a dataframe column also has a name 
* A numpy array can also be indexed but you can't change what the indecies are named 
    (always indexed in the same way)
* tables can have columns of all different data types 

In [ ]:
np_array = np.array([1,2,3,4,5])

print('Numpy Array')
print(np_array)
print('\n')

#It will assign the index for you if you don't give it one
#Indexing always starts at 0 in python
pd_series = pd.Series([1,2,3,4,5])

print('Pandas Series')
print(pd_series)

### Series in Pandas

In [ ]:
#You can define the index for the series 
s1 = pd.Series([1,2,3,4,5],index = ['a','b','c','d','e'])

print(s1, '\n')

#building a series from a dictionary 
data = {'cats' : 15., 'dogs' : 35., 'turtles' : 100.}
s2 = pd.Series(data)

print(s2, '\n')

#You can assign the index (pandas handels missing data very well)
s3 = pd.Series(data,index=['turtles','cats','birds','dogs'])

print(s3, '\n')

### Dataframes in Pandas

In [ ]:
#example of how to build a dataframe
#Example: I have an array of data that tells me the number of animals on a given date
dates = pd.date_range('20180620', periods=6)
rand_data = np.random.randint(low=0, high=100, size=(6,4))
df = pd.DataFrame(rand_data, index=dates, columns=['cats','dogs','birds','turtles'])

print(df)
print('\n')

#If you don't care what the index is you don't have to give it one
#If you don't assign and index it just gives them index values from 0 to # rows

df = pd.DataFrame(rand_data, columns=['cats','dogs','birds','turtles'])

print(df)

## Importing data with Pandas 
* Pandas can easily read in files of data in one line 
* Unless you specify the dtypes you want for each column it chooses the best match 
* Each read function in Pandas is very flexible with a lot of arguments to to read in a lot of differnet formats
* Generic delimited text [(documentation)](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_table.html)
    * `read_csv()` 
    * `read_table()` - defaults to tab separated

### Other formats
Pandas has built-in support for [reading from or to other common formats/sources](http://pandas.pydata.org/pandas-docs/stable/io.html):
* Excel -- `read_excel()`
* JSON -- `read_json()`
* SQL -- `read_sql()`
* Stata -- `read_stata()`
* SAS (XPORT or SAS7BDAT) -- `read_sas()`
* etc...

## We are going to work with the HETDEX Pilot Survey emission line catalog
* this is a Vizier catalog that can be found [here](http://vizier.u-strasbg.fr/viz-bin/VizieR-3?-source=J/ApJS/192/5/hetdex)
* The page has an option to save the table as a tab separated file which I did 

In [ ]:
#importing data with pandas 

#Because the file has such a large header you need to tell it which row you want it to use for the column names (header)
#Tell it to skip the rows below it that have the units and the line 
#If you don't care about the header information it is sometimes easier to just delete it from the top of the file
#  if you delete the header just make sure to leave the top row of column names
hps_pd = pd.read_table('hps_catalog.txt', delimiter=None, header=62, skiprows=[64,65])
hps_pd

### Astropy supports different file formats (commonly used in astronomy) that Pandas doesn't
* some of the formats are more astronomy specific (ex. fits)
* It does allow you to save tables as deluxe latex tables which is extremely useful!
* It also will read in a row of units for columns (useful when reading in catalogs)
* A [list of the format Astropy supports](http://docs.astropy.org/en/stable/io/unified.html#built-in-readers-writers)

In [ ]:
#importing data with astropy tables

hps_at = Table.read('hps_catalog.txt', format='ascii')
hps_at


## Reading in with Pandas vs. Astropy Tables 
* You can see that Tables did not automatically pick up on there being two rows under the column names that are not data
* the Tables read function does not have arguments for formating files like pandas 
* so unless your data fits the formats provided astropy tables can be annoying to work with 
* I usually have to format the file before I try to read it in with astropy tables 
* Pandas provides a lot more flexlibilty 

## However, you can also load it in with Astropy Tables using Astroquery
* Astroquery allows you to load in a catalog from a bunch of different sources 
* [Here](https://astroquery.readthedocs.io/en/latest/#catalog-archive-and-other) is a list of all of the catalogs and archives that now work with astroquery (some are better supported than others)
* [Here](http://astroquery.readthedocs.io/en/latest/gallery.html) is a page with astroquery examples for some of the databases 

In [ ]:
#queries with astropy (Vizier)

hps_catalog_list = Vizier.find_catalogs('J/ApJS/192/5/')
hps_catalogs = Vizier.get_catalogs(hps_catalog_list.keys())
hps_at = hps_catalogs[0]

hps_at
#hps_at.show_in_notebook() #This is a tool for viewing astropy tables in a Jupyter notebook
#hps_at.show_in_browser(jsviewer=True) #this is a tool for viewing a searchable astropy table in a web browser

## Interfacing between pandas and astropy tables 
* you can covert from one to the other very easily with the following commands 

In [ ]:
# astropy tables to pandas
hps_pd_from_at = hps_at.to_pandas()

# pandas to astropy table
hps_at_from_pd = Table.from_pandas(hps_pd)

# **From now on in this tutorial we will be using Pandas**
* some of this functionality exists in Astropy Tables but not all of it. 

## Exploring your data set 

In [ ]:
#hps_pd.head()
hps_pd.tail(10)

In [ ]:
hps_pd.columns

In [ ]:
hps_pd.columns.values

In [ ]:
hps_pd.dtypes

In [ ]:
hps_pd.describe()

## Indexing Pandas dataframes
* You can either index by the row and column number as you with in numpy 
* Or more commonly you use the columns names and the row index value
* Pandas defults to using label indexing
* For clarity pandas makes you specify if you using integer or label indexing
    * this is neccessary because the columns names are allowed to be numbers 
    * without specifying pandas would not know if you mean column number 1 or the column named 1 

In [ ]:
#label indexing

#hps_pd['EW']
#hps_pd.EW
hps_pd.loc[:,'EW']

In [ ]:
#You can ask for certian rows in that column
hps_pd['EW'][2:5] 

In [ ]:
#can also get more than one column
hps_pd.loc[:25,['Trans','Lya','S/N']]

In [ ]:
#I want all of the detections at a z<0.5 with and EW>50

#You can also use the query function
#hps_pd.query('(z < 0.5) & (EW > 50.)')

#can also write it this way 
hps_pd.loc[(hps_pd.z < 0.5) & (hps_pd.EW > 50)]


In [ ]:
#You can also index by location like you do in numpy using iloc

#If you give it a single value it returns a series of that row number 
hps_pd.iloc[0]

In [ ]:
# You can also give it a specific section of the table you want 
#format is [row_start:row_end, col_start:col_end]
hps_pd.iloc[0:10, 2:6]

In [ ]:
#It will also take a list of column numbers 
hps_pd.iloc[0:10, [1,4,6,9]]

## Useful Functions
* Pandas has a lot of useful built in functions 
* Pandas apply function allows you to operate a function on all values in a column/table

In [ ]:
hps_pd['Trans'].values

In [ ]:
#You may notice when it read in the Trans column it added extra spaces making it difficult to query
# You can use the apply function to strip each value in the Trans column 
# Anonymous (lambda) funcitons in python are a convienent way to write a one line function to operate over a variable
# the strip() function strips a string of its whitespace
hps_pd['Trans'] = hps_pd['Trans'].apply(lambda x: x.strip())

#can ues the values function to return an array of just a columns values 
hps_pd['Trans'].values

In [ ]:
#find the unique types of emission line types 
emission_lines = hps_pd['Trans'].unique()
emission_lines

In [ ]:
#get an table of HPS ids, fluxes, and S/N for Ly{alpha} lines where the probablilty of it being Lya is greater than 50%
hps_pd.query('(Trans == "Ly{alpha}") & (Lya > 0.5)').loc[:,['HPS','Flux','S/N']]

In [ ]:
# you want the lya line with the largest flux 
max_lya_flux = hps_pd.query('(Trans == "Ly{alpha}") & (Lya > 0.5)')['Flux'].max()
print('MAX Flux: ', max_lya_flux, '\n')

max_flux_obj = hps_pd.loc[hps_pd['Flux'] == max_lya_flux]
print(max_flux_obj, '\n')
#hps_pd.iloc[max_lya_flux]

In [ ]:
#want to know how many of each line
hps_pd['Trans'].value_counts()

In [ ]:
#count the number of LyA lines with a probablility greater than 50%
hps_pd.query('(Trans == "Ly{alpha}") & (Lya > 0.5)')['HPS'].count()

In [ ]:
#sort the LyA lines by the flux values in descending order 
LyA_sorted_by_flux = hps_pd.query('Trans == "Ly{alpha}"').sort_values('Flux', ascending=False)
LyA_sorted_by_flux

In [ ]:
#build function to give to apply for a column
def AA_to_nm(wl):
    wl = wl/10
    return wl

hps_pd['lambda'].apply(AA_to_nm)

In [ ]:
#Here is a function that builds all of the RAj2000 and DEJ2000 values into actual astropy coordinates
#It stores the coordinates in a new column in the dataframe called coord

def build_astropy_coords(row):
    ra  = row['RAJ2000'].split()
    dec = row['DEJ2000'].split()
    c = SkyCoord(ra[0]+'h'+ra[1]+'m'+ra[2]+'s', dec[0]+'d'+dec[1]+'m'+dec[2]+'s', frame='icrs')
    return c

#You give apply the function and the argument axis=1 to it feeds the function the rows of the dataframe
#It stores these new values in a new column called coord
hps_pd['coord'] = hps_pd.apply(build_astropy_coords, axis=1)

#prints the new ra and dec of the stored coordinate for the first row
#print(hps_pd['coord'][0].ra, hps_pd['coord'][0].dec)

hps_pd.head()

In [ ]:
#want to find all of the rows that have an x-ray detection 

#first want to replace all of the rows with nothing but spaces in the X-ray column with nan values 
hps_pd['X-ray'] = hps_pd['X-ray'].replace('                ', np.nan, regex=True)

#use the drapna function to remove the rows without a detection. 
#the .index function returns the indecies of the columns with an x-ray detection
#Then use the .loc function to find the rows with that index 
possible_AGN = hps_pd.loc[hps_pd['X-ray'].dropna().index]
possible_AGN

In [ ]:
#the dropna function had many arguments that make it very powerful. 

hps_drop_allNaN  = hps_pd.dropna()     #drop all rows that have any NaN values
hps_drop_onlyNaN = hps_pd.dropna(how='all')     #drop only if ALL columns are NaN
hps_drop_someNaN = hps_pd.dropna(thresh=3)   #Will only keep rows with 3 or more **not** NaN values

#this is a better way of writing the line defining possible_AGN in the previous cell
possible_AGN = hps_pd.dropna(subset=['X-ray'])   #Drop only if NaN in specific column 
possible_AGN

In [ ]:
#changing/appending data 
#saw already how to rename columns but here is how to add/remove columns 

## Building groups
* you can build groups based on column values 
* This could be really useful in this example to group by the line type (Trans column)
* pandas provides many built-in operations on groups--e.g., mean, count, etc.
* see http://pandas.pydata.org/pandas-docs/stable/groupby.html#groupby-object-attributes

In [ ]:

# Split the dataset on Trans. This creates a pandas GroupBy object.
lines = hps_pd.groupby('Trans')

# You can call one of the groups by the name
NeV_lines = lines.get_group("[NeV]3426")
print(NeV_lines)

lines['Flux'].mean()

# The above line can also be more explicitly written as:
# groups['age'].aggregate('mean')

## Types of joins
* Joins allow you to join together different dataframes based on a key values (similar to SQL)
* for more information see the [merging documentation](https://pandas.pydata.org/pandas-docs/stable/merging.html)
* `left`  joins - Use keys from left frame only
* `right` joins - Use keys from right frame only
* `outer` joins - Use union of keys from both frames
* `inner` joins - Use intersection of keys from both frames

#### Diagram of joins
<img src="https://i.stack.imgur.com/hMKKt.jpg">

In [ ]:
#Load in new a table from a paper to merge with the HPS catalog
#Bridge et. al. 2014 derived properties from SED fitting for the OII emitters in the HPS catalog

bridge_cols=['HPS_name', 'logM', 'logSFR', 'E(B-V)', 'OII_z']
bridge = pd.read_table('bridge2015_cat.ascii', delim_whitespace=True, names=bridge_cols, skiprows=16)
bridge

In [ ]:
#We want to join this table with the HPS catalog
#They share the common key (HPS id number) but they have different names in each table

#We first change the name of the HPS id in the bridge dataframe
bridge.rename(columns={'HPS_name':'HPS'}, inplace=True)

#We can then merge the two dataframes on the key HPS
#We use and inner join so that we only get rows back that have both HPS and bridge data
#If we use any other join it would fill in the missing data with NaNs 
HPS_OII = pd.merge(hps_pd, bridge, how='inner', on=['HPS'])
HPS_OII

#### Here is an example of a outer join
* In this case we want to preserve all of the rows in the HPS catalog 
* but when we can we want to add a value for the field that emission line was found in

In [ ]:
#read in the table from the vizier catalog that contains the information on the fields observed 
hps_fields = pd.read_table('hps_fields.txt', delimiter=None, header=50, skiprows=[52,53])
print(hps_fields)

In [ ]:
#this function just finds the hour of the RA in the column 'RAJ2000'
def find_RA_hour(row):
    ra_h = row['RAJ2000'].split()[0] 
    return ra_h

#apply this function to build an RA_hour column in the hps dataframe
hps_pd['RA_hour'] = hps_pd.apply(find_RA_hour, axis=1)
print(hps_pd['RA_hour'].unique())

#apply this function to build an RA_hour column in the hps_fields dataframe
hps_fields['RA_hour'] = hps_fields.apply(find_RA_hour, axis=1)

#we only want to add the field name to the hps dataframe so this is just the Field column and the RA_hour to merge on
fields = hps_fields.loc[:,['Field','RA_hour']]

#We can then join the two databases to add the field column to the hps dataframe
hps_join_fields = pd.merge(hps_pd, fields, how='outer', on=['RA_hour'])
hps_join_fields.head(10)

### Creating crosstabs 
* This is a useful tool that lets you analzye columns against eachother 

In [ ]:
#You want to know how many of each line is in each field

#creating crosstabs from groups
ctab = pd.crosstab(hps_join_fields['Trans'], hps_join_fields['Field'])

#You can also normalize the crosstab
#ctab = pd.crosstab(hps_join_fields['Trans'], hps_join_fields['Field'], normalize='index')

order = ctab.sum(1).sort_values(ascending=False).index
ctab = ctab.loc[order, :]

ctab

## Saving dataframes 
* This can be done in one line with both pandas and astropy tables 
* For the most part the format of the output does not matter since it is just going to be loaded in as a dataframe later
* in Pandas you can save the dataframe as a .csv or pickle file (may be more efficient for large files) 
* There are a lot more options for astropy tables 
    * basically any file you can load in you can save that file as the any of those outputs
    * the most useful thing is you can save a datafame as a latex deluxe table 

In [ ]:
#saving dataframes in Pandas 
HPS_OII.to_csv('bridge_HPS_cat.csv')

#saving astropy table as a deluxe latex table 
#Here is an example of how we could make a table of just the possible AGN
hps_at_OII = hps_at[hps_at['Trans'] == '[OII]']
hps_at_OII.write('HPS_OII_table.tex', format='ascii.aastex')

## Pandas makes it easy to do statistics on columns and the entire dataframe
* The describe function we saw earlier is customizable 

In [ ]:
#statistics with panadas 

print(hps_pd.query('Trans == "[OII]"')['EW'].mean())
print(hps_pd.query('Trans == "[OII]"')['EW'].std())

# Describe the continuous variables in the dataset.
# We can also include categoricals, or select other percentiles.
#hps_pd.describe(percentiles=[0.1, 0.9]).round(2)

#lines.get_group('[OII]').describe()

## Data visualization using Pandas dataframes

### Easy plotting with Matplotlib using Pandas columns

In [ ]:
#It is easy to plot in matplotlib by giving it pandas columns 

# Scatterplot of mass vs. SFR for the OII emitters from the Bridge et. al. 2014 paper
plt.scatter(HPS_OII['logM'], HPS_OII['logSFR'], color='green', s=30)
plt.xlabel("log(M)")
plt.ylabel("log(SFR)")
plt.title("Mass vs. SFR for the OII emitters in HPS", fontsize=16);

### Plotting in Seaborn with dataframes

In [ ]:
#The crosstab we made can be easily plotted as a headmap 
ax = sns.heatmap(ctab[:12], annot=True, fmt='d', linewidths=.5,
                 annot_kws={'size': 14}, cmap='Blues')

# Customize the label size and figure size in base matplotlib.
# Notice we can access all axis and figure properties through
# # the handle we saved when calling seaborn.
ax.tick_params(labelsize=14)
ax.figure.set_size_inches((7, 8))  # update the plot size

In [ ]:
# Plot a heatmap of pairwise correlations between all continuous variables
r = HPS_OII.corr().round(2)

with sns.plotting_context("notebook", font_scale=1.1):
    ax = sns.heatmap(r, annot=True)
    ax.figure.set_size_inches((12, 9.5))

In [ ]:
#pairplots from a select group of columns for the OII catalog

tmp_data = HPS_OII.loc[:,['Flux', 'logM', 'logSFR', 'E(B-V)','z']]

with sns.plotting_context("notebook", font_scale=1.3):  # Make text bigger
    sns.pairplot(tmp_data, diag_kind='hist', kind='reg')

In [ ]:
g = sns.jointplot(x="logM", y="logSFR", data=HPS_OII, kind="kde", color="purple")
g.plot_joint(plt.scatter, c="white", s=30, linewidth=1, marker="+")
g.ax_joint.collections[0].set_alpha(0)
g.set_axis_labels("$logM$", "$logSFR$");

In [ ]:
f, ax = plt.subplots(figsize=(6, 6))
cmap = sns.cubehelix_palette(as_cmap=True, dark=0, light=1, reverse=True)
g = sns.kdeplot(HPS_OII.logM, HPS_OII.logSFR, cmap=cmap, n_levels=60, shade=True)

### Plotting in Pandas 

In [ ]:
HPS_OII.plot.scatter(x='logM', y='logSFR')

In [ ]:
HPS_OII.plot.scatter(x='logM', y='logSFR', c='E(B-V)', s=50)

In [ ]:
#Only want to plot histogram for lines with more than 15 detections
many_detections = hps_pd.groupby('Trans').filter(lambda x: len(x)>15)

# Plot flux histograms for all line types--with shared x-axis
many_detections['Flux'].hist(by=many_detections['Trans'], bins=20, sharex=True);

# Embiggen the figure a bit
plt.gcf().set_size_inches((10, 10))

In [ ]:
line_counts = hps_pd['Trans'].value_counts()
print(line_counts)

many_.plot.pie(subplots=True, autopct='%.2f', fontsize=15, figsize=(8, 8))